In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
G = nx.DiGraph() # Grafo direcionado

In [ ]:
data = "dataset"

for file in os.listdir(data):
    if file.endswith(".edges"):
        file_path = os.path.join(data, file)

        with open(file_path, "r") as f:
            for line in f:
                try:
                    node1, node2 = line.split()
                    G.add_edge(node1, node2) # Adiciona a aresta ao grafo entre o nó e o nó de conexão
                except ValueError:
                    print(f"Erro ao processar a linha: {line.strip()}")
                    continue

print(f"Grafo criado com {G.number_of_nodes()} nós e {G.number_of_edges()} arestas.")
# Salva o grafo em formato GEXF no diretorio atual do projeto
nx.write_gexf(G, "grafo.gexf")

In [ ]:
import random

G_sample = G.subgraph(random.sample(list(G.nodes()), 5000)) # Cria um subgrafo com 5000 nós aleatórios
print(f"Grafo amostrado com {G_sample.number_of_nodes()} nós e {G_sample.number_of_edges()} arestas.")


In [ ]:
# encontrar os nós mais influentes
pagerank = nx.pagerank(G_sample, alpha=0.85)
sorted_pagerank = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)
top_10 = sorted_pagerank[:10]
print("Top 10 nós com maior PageRank:")
for i, (node, score) in enumerate(top_10):
    print(f"{i+1}. Nó: {node}, PageRank: {score:.4f}")

In [ ]:
# Visualização dos nós mais influentes
plt.figure(figsize=(14, 12))

# Preparar o layout do grafo
pos = nx.spring_layout(G_sample, seed=42)

# Definir tamanhos dos nós baseados no PageRank
node_sizes = []
node_colors = []

# Identificar os top 10 nós
top_nodes = [node for node, _ in sorted_pagerank[:10]]

for node in G_sample.nodes():
    if node in top_nodes:
        node_sizes.append(1000 * pagerank[node] / max(pagerank.values()))
        node_colors.append('red')
    else:
        node_sizes.append(300 * pagerank.get(node, 0) / max(pagerank.values()))
        node_colors.append('skyblue')

# Desenhar o grafo
nx.draw_networkx_edges(G_sample, pos, alpha=0.2, edge_color='gray', width=2)
nx.draw_networkx_nodes(G_sample, pos, node_size=node_sizes, node_color=node_colors, alpha=0.8)

# Adicionar labels apenas para os nós mais influentes
labels = {node: node for node in top_nodes}
nx.draw_networkx_labels(G_sample, pos, labels=labels, font_size=12, font_weight='bold')

plt.title("Visualização dos Nós Mais Influentes (PageRank)", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Girvan-Newman
temp = G_sample.to_undirected()
comunidades = nx.algorithms.community.girvan_newman(temp)

grupos = []

for com in next(comunidades):
    grupos.append(list(com))

print(f"Foram detectadas {len(grupos)} comunidades pelo algoritmo Girvan-Newman")

In [ ]:
color_palette = sns.color_palette("hsv", len(grupos))
color_map = {i: color_palette[i] for i in range(len(grupos))}

cores = []
for vertice in G_sample:
    for idx, grupo in enumerate(grupos):
        if vertice in grupo:
            cores.append(color_map[idx])
            break
    else:
        cores.append("gray")

plt.figure(figsize=(14, 12))
nx.draw_networkx_edges(G_sample, pos, alpha=0.2, edge_color='red', width=2)
nx.draw_networkx_nodes(G_sample, pos, node_size=50, node_color=cores, alpha=0.8)
plt.title("Visualização das comunidades (Girvan-Newman)", fontsize=16)
plt.axis('off')
plt.show()


In [ ]:
# Louvain
partition = nx.algorithms.community.louvain_communities(G_sample, seed=42)

print(f"Foram detectadas {len(partition)} comunidades pelo algoritmo Louvain")

In [ ]:
# Mostrando as comunidades detectadas pelo algoritmo Louvain
node_to_community = {}
for community_index, community in enumerate(partition):
    for node in community:
        node_to_community[node] = community_index

# Generate a list of colors for each node based on its community
cores = [node_to_community[node] for node in G_sample.nodes]

pos = nx.spring_layout(G_sample, seed=42)

plt.figure(figsize=(14, 12))
nx.draw_networkx_edges(G_sample, pos, alpha=0.2, edge_color='red', width=2)
nx.draw_networkx_nodes(G_sample, pos, node_size=50, node_color=cores, alpha=0.8, cmap=plt.cm.rainbow)
plt.title("Visualização das comunidades (Louvain)", fontsize=16)
plt.axis('off')
plt.show()

In [ ]:
import heapq

# Usar um heap para manter apenas os top_n nós
heap = []
top_n = 10

for node in G_sample.nodes():
    grau_saida = G_sample.out_degree(node)
    if len(heap) < top_n:
        heapq.heappush(heap, (grau_saida, node))
    elif grau_saida > heap[0][0]:
        heapq.heapreplace(heap, (grau_saida, node))

top_compartilhadores = [(node, grau) for grau, node in sorted(heap, reverse=True)]

print("Os 10 usuários que mais compartilham conteúdos:")
for i, (node, out_degree) in enumerate(top_compartilhadores, 1):
    print(f"{i}. Nó {node}: {out_degree} compartilhamentos")

In [ ]:
# Visualizar os usuários que mais compartilham com um gráfico de barras
plt.figure(figsize=(14, 12))

nodes = [node for node, _ in top_compartilhadores]
degrees = [degree for _, degree in top_compartilhadores]

# Criar gráfico de barras
bars = plt.bar(range(len(nodes)), degrees, color='skyblue', edgecolor='navy')

# Adicionar valores no topo das barras
for i, (bar, degree) in enumerate(zip(bars, degrees)):
    plt.text(bar.get_x() + bar.get_width()/2, degree + 0.1, 
             str(degree), ha='center', va='bottom', fontweight='bold')

plt.xlabel('Usuários', fontsize=14)
plt.ylabel('Número de Compartilhamentos', fontsize=14)
plt.title('Usuários que Mais Compartilham Conteúdos', fontsize=16)
plt.xticks(range(len(nodes)), [f"Nó {node}" for node in nodes], rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# calculando usuarios ponte
betweenness = nx.betweenness_centrality(G_sample, normalized=True)
sorted_betweenness = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)

print("Os 10 usuários que mais atuam como ponte entre comunidades:")
for i, (node, score) in enumerate(sorted_betweenness[:10], 1):
    print(f"{i}. Nó {node}: {score:.4f}")

In [ ]:
plt.figure(figsize=(14, 12))
pos = nx.spring_layout(G_sample, seed=42)
top_nodes = [node for node, _ in sorted_betweenness[:10]]

tamanhos = []
cores = []

maximo_betweenness = max(betweenness.values()) if betweenness else 1
if maximo_betweenness == 0:
    print("Eita")
    maximo_betweenness = 1

for node in G_sample.nodes():
    if node in top_nodes:
        tamanhos.append((1000 * betweenness[node])/ maximo_betweenness)
        cores.append('red')
    else:
        tamanhos.append((300 * betweenness.get(node, 0))/ maximo_betweenness)
        cores.append('skyblue')

# Desenhar o grafo
nx.draw_networkx_edges(G_sample, pos, alpha=0.2, edge_color='red', width=2)
nx.draw_networkx_nodes(G_sample, pos, node_size=tamanhos, node_color=cores, alpha=0.8)
# Adicionar labels apenas para os nós mais influentes
labels = {node: node for node in top_nodes}
nx.draw_networkx_labels(G_sample, pos, labels=labels, font_size=12, font_weight='bold')
plt.title("Visualização dos Nós que Mais Atuaram como Ponte entre Comunidades", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()



In [ ]:
temp = G_sample.to_undirected()

distancias = {}

for node in temp.nodes():
    try:
        caminhos = nx.single_source_dijkstra_path_length(temp, node)
        if len(caminhos) > 0:
            distancias[node] = len(temp) - 1 / sum(caminhos.values())
        else:
            distancias[node] = 0
    except nx.NetworkXError:
        distancias[node] = 0

distancias = sorted(distancias.items(), key=lambda x: x[1], reverse=True)
print("Os 10 usuários mais centrais:")
for i, (node, score) in enumerate(distancias[:10], 1):
    print(f"{i}. Nó {node}: {score:.4f}") 

In [ ]:
plt.figure(figsize=(14, 12))
pos = nx.spring_layout(G_sample, seed=42)
top_nodes = [node for node, _ in distancias[:10]]
tamanhos = []
cores = []

maximo_distancia = max([score for _, score in distancias]) if distancias else 1

if maximo_distancia == 0:
    maximo_distancia = 1

# Convert distancias to a dictionary for easier access
distancias_dict = dict(distancias)

for node in G_sample.nodes():
    if node in top_nodes:
        tamanhos.append((0.4 * distancias_dict.get(node, 0)) + 1 / maximo_distancia)
        cores.append('red')
    else:
        tamanhos.append((0.06 * distancias_dict.get(node, 0)) + 1 / maximo_distancia)
        cores.append('skyblue')

# Desenhar o grafo
nx.draw_networkx_edges(G_sample, pos, alpha=0.2, edge_color='red', width=2)
nx.draw_networkx_nodes(G_sample, pos, node_size=tamanhos, node_color=cores, alpha=0.8)
# Adicionar labels apenas para os nós mais influentes
labels = {node: node for node in top_nodes}
nx.draw_networkx_labels(G_sample, pos, labels=labels, font_size=12, font_weight='bold')
plt.title("Visualização dos Nós Mais Centrais", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()